## First commands

In [156]:
cd /home/jovyan/training_bacterial_comparative_genomics/jupyter

(base) 

: 1

In [157]:
ls

Trainings_genomics_june2022.ipynb
(base) 

: 1

In [159]:
mkdir assembly

(base) 

: 1

In [5]:
ls

assembly  Trainings_genomics_june2022.ipynb


In [160]:
mkdir raw_data

(base) 

: 1

In [158]:
wget http://bioinfo-web.mpl.ird.fr/ideogram/CIX4108.sample.fastq.gz

--2022-06-13 13:13:53--  http://bioinfo-web.mpl.ird.fr/ideogram/CIX4108.sample.fastq.gz
Resolving bioinfo-web.mpl.ird.fr (bioinfo-web.mpl.ird.fr)... 91.203.35.161
Connecting to bioinfo-web.mpl.ird.fr (bioinfo-web.mpl.ird.fr)|91.203.35.161|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 253558784 (242M) [application/x-gzip]
Saving to: ‘CIX4108.sample.fastq.gz’

CIX4108.sample.fast 100%[===================>] 241.81M  10.8MB/s    in 20s     

2022-06-13 13:14:13 (11.9 MB/s) - ‘CIX4108.sample.fastq.gz’ saved [253558784/253558784]

(base) 

: 1

In [161]:
mv CIX4108.sample.fastq.gz raw_data

(base) 

: 1

#### How many reads are there in the raw Fastq file?

In [162]:
zcat raw_data/CIX4108.sample.fastq.gz | wc -l


gzip: raw_data/CIX4108.sample.fastq.gz: unexpected end of file
68057
(base) 

: 1

## Genome Assembly (from Oxford Nanopore Technologies (ONT) long reads) (using Flye)

In [163]:
conda create -n flye -c bioconda flye

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.12.0
  latest version: 4.13.0

Please update conda by running

    $ conda update -n base conda



## Package Plan ##

  environment location: /opt/conda/envs/flye

  added / updated specs:
    - flye


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    flye-2.9                   |   py39h6935b12_1        35.2 MB  bioconda
    openssl-3.0.3              |       h166bdaf_0         2.9 MB  conda-forge
    python-3.9.13              |h2660328_0_cpython        26.7 MB  conda-forge
    python_abi-3.9             |           2_cp39           4 KB  conda-forge
    setuptools-62.3.4          |   py39hf3d152e_0         1.3 MB  conda-forge
    tzdata-2022a               |       h191b570_0         121 KB  conda-forge
    ------------------------------------------------------------
                       

In [ ]:
conda activate flye

In [6]:
flye --nano-raw /data2/formation/Bacterial_genomics/raw_data/CIX4108.sample.fastq -o assembly >>flye.log 2>&1

: 127

In [7]:
cp -rf ../data/precomputed_assembly/assembly_precomputed.fasta assembly

In [9]:
grep -c ">" assembly/assembly_precomputed.fasta

2


## Separate chromosomal and plasmid scaffolds (using MOB-Suite)

In [11]:
singularity exec /home/jovyan/mob_suite_3.0.3.sif mob_recon -i assembly/assembly_precomputed.fasta -o assembly.mob_recon >>mob_recon.log 2>&1

: 255

In [174]:
cp -rf ../data/mob_recon assembly.mob_recon

(base) 

: 1

In [13]:
ls assembly.mob_recon

chromosome.fasta  contig_report.txt  mobtyper_results.txt  plasmid_AD399.fasta


In [14]:
more assembly.mob_recon/contig_report.txt

sample_id	molecule_type	primary_cluster_id	secondary_cluster_id	contig_id	size	gc	md5	circularity_status	rep_type(s)	rep_type_accession(s)	relaxase_type(s)	relaxase_type_accession(s)	mpf_type	mpf_type_accession(s)	orit_type(s)	orit_accession(s)	predicted_mobility	mash_nearest_neighbor	mash_neighbor_distance	mash_neighbor_identification	repetitive_dna_id	repetitive_dna_type	filtering_reason
assembly_precomputed	chromosome	-	-	contig_1_circular_rotated	4682534	63.833983907004196	cf955654136e9041810a80b8191fa41f	not tested	-	-	MOBP	NC_007507_00032	-	-	-	-	-	-	-	-	-	-	-
assembly_precomputed	plasmid	AD399	-	contig_2_circular_rotated	69058	61.861044339540676	475037e138b4ade1836aa05f421c47b8	not tested	rep_cluster_1289	000607__CP000620_00033	MOBP	CP022994_00148	-	-	-	-	-	CP033195	0.0537328	Xanthomonas oryzae pv. oryzae	-	-	-


## Genome annotation (using Prokka)

In [ ]:
conda create -n prokka -c conda-forge -c bioconda prokka

In [45]:
conda activate prokka

(prokka) 

: 1

In [46]:
prokka assembly.mob_recon/chromosome.fasta --prefix assembly --force --outdir prokka_out >>prokka.log 2>&1

(prokka) 

: 1

In [51]:
conda deactivate

(base) 

: 1

In [52]:
head -40 prokka_out/assembly.gbk

LOCUS       contig_1_circular_rotated4682534 bp   DNA  linear       10-JUN-2022
DEFINITION  Genus species strain strain.
ACCESSION   
VERSION
KEYWORDS    .
SOURCE      Genus species
  ORGANISM  Genus species
            Unclassified.
COMMENT     Annotated using prokka 1.14.6 from
            https://github.com/tseemann/prokka.
FEATURES             Location/Qualifiers
     source          1..4682534
                     /organism="Genus species"
                     /mol_type="genomic DNA"
                     /strain="strain"
     CDS             1..1329
                     /gene="dnaA"
                     /locus_tag="GBGACGDJ_00001"
                     /inference="ab initio prediction:Prodigal:002006"
                     /inference="similar to AA sequence:UniProtKB:P03004"
                     /codon_start=1
                     /transl_table=11
                     /product="Chromosomal replication initiator protein DnaA"
                     /db_xref="COG:COG0593"
              

: 1

In [49]:
head -10 prokka_out/assembly.gff

##gff-version 3
##sequence-region contig_1_circular_rotated 1 4682534
contig_1_circular_rotated	Prodigal:002006	CDS	1	1329	.	+	0	ID=GBGACGDJ_00001;Name=dnaA;db_xref=COG:COG0593;gene=dnaA;inference=ab initio prediction:Prodigal:002006,similar to AA sequence:UniProtKB:P03004;locus_tag=GBGACGDJ_00001;product=Chromosomal replication initiator protein DnaA
contig_1_circular_rotated	Prodigal:002006	CDS	1607	2389	.	+	0	ID=GBGACGDJ_00002;Name=dnaN_1;db_xref=COG:COG0592;gene=dnaN_1;inference=ab initio prediction:Prodigal:002006,similar to AA sequence:UniProtKB:Q9I7C4;locus_tag=GBGACGDJ_00002;product=Beta sliding clamp
contig_1_circular_rotated	Prodigal:002006	CDS	2346	2708	.	+	0	ID=GBGACGDJ_00003;Name=dnaN_2;db_xref=COG:COG0592;gene=dnaN_2;inference=ab initio prediction:Prodigal:002006,similar to AA sequence:UniProtKB:P0A988;locus_tag=GBGACGDJ_00003;product=Beta sliding clamp
contig_1_circular_rotated	Prodigal:002006	CDS	3777	4883	.	+	0	ID=GBGACGDJ_00004;Name=recF;db_xref=COG:COG1195;gene=recF;

: 1

In [50]:
grep COG prokka_out/assembly.gff | head -5

contig_1_circular_rotated	Prodigal:002006	CDS	1	1329	.	+	0	ID=GBGACGDJ_00001;Name=dnaA;db_xref=COG:COG0593;gene=dnaA;inference=ab initio prediction:Prodigal:002006,similar to AA sequence:UniProtKB:P03004;locus_tag=GBGACGDJ_00001;product=Chromosomal replication initiator protein DnaA
contig_1_circular_rotated	Prodigal:002006	CDS	1607	2389	.	+	0	ID=GBGACGDJ_00002;Name=dnaN_1;db_xref=COG:COG0592;gene=dnaN_1;inference=ab initio prediction:Prodigal:002006,similar to AA sequence:UniProtKB:Q9I7C4;locus_tag=GBGACGDJ_00002;product=Beta sliding clamp
contig_1_circular_rotated	Prodigal:002006	CDS	2346	2708	.	+	0	ID=GBGACGDJ_00003;Name=dnaN_2;db_xref=COG:COG0592;gene=dnaN_2;inference=ab initio prediction:Prodigal:002006,similar to AA sequence:UniProtKB:P0A988;locus_tag=GBGACGDJ_00003;product=Beta sliding clamp
contig_1_circular_rotated	Prodigal:002006	CDS	3777	4883	.	+	0	ID=GBGACGDJ_00004;Name=recF;db_xref=COG:COG1195;gene=recF;inference=ab initio prediction:Prodigal:002006,similar to AA sequence:

: 1

## GC content analysis (SkewIT)

In [53]:
git clone https://github.com/jenniferlu717/SkewIT.git

Cloning into 'SkewIT'...
remote: Enumerating objects: 123, done.
remote: Counting objects: 100% (123/123), done.
remote: Compressing objects: 100% (94/94), done.
remote: Total 123 (delta 40), reused 94 (delta 26), pack-reused 0
Receiving objects: 100% (123/123), 683.64 KiB | 4.14 MiB/s, done.
Resolving deltas: 100% (40/40), done.
(base) 

: 1

In [55]:
python SkewIT/src/gcskew.py -i assembly.mob_recon/chromosome.fasta -o gcskew.txt -k 500

PROGRAM START TIME: 06-10-2022 17:10:45
    Input file: assembly.mob_recon/chromosome.fasta
    Output: gcskew.txt
    Window size (bp): 500
    Frequency (bp): 500
>> Reading sequence file assembly.mob_recon/chromosome.fasta
	1 seqs found
>> Calculating/Saving GC Skew for 1 seqs
	Finished calculating gc skew for 1 sequences
PROGRAM FINISH TIME: 06-10-2022 17:10:45
(base) 

: 1

In [61]:
head -5 gcskew.txt

Sequence	Index	GC Skew (0kb)
contig_1_circular_rotated	0	-0.05952381
contig_1_circular_rotated	500	0.00958466
contig_1_circular_rotated	1000	-0.00327869
contig_1_circular_rotated	1500	-0.01010101
(base) 

: 1

In [62]:
grep -v 'Sequence' gcskew.txt | awk {'print "Chr "$2+1" "$2+5000" "$3'}  >gcskew.circos.txt

(base) 

: 1

In [63]:
awk {'if ($3 == "CDS" && $7 == "+")print "Chr "$4" "$5'} prokka_out/assembly.gff >genes_plus.txt
awk {'if ($3 == "CDS" && $7 == "-")print "Chr "$4" "$5'} prokka_out/assembly.gff >genes_minus.txt

(base) (base) 

: 1

## Visualize genome annotation (using Circos)

In [133]:
git clone https://github.com/vigsterkr/circos.git
cd circos
./install-unix

Cloning into 'circos'...
remote: Enumerating objects: 885, done.
remote: Total 885 (delta 0), reused 0 (delta 0), pack-reused 885
Receiving objects: 100% (885/885), 39.85 MiB | 17.96 MiB/s, done.
Resolving deltas: 100% (184/184), done.
(base) 

: 1

In [64]:
echo "chr - Chr 1 0 4600000 black" >karyotype.txt

(base) 

: 1

In [ ]:
cp -rf ../data/circos1.conf /home/jovyan/training_bacterial_comparative_genomics/jupyter/circos/circos1.conf

#### Edit the Circos configuration file to adapt and customize your Circos image

In [140]:
cd /home/jovyan/training_bacterial_comparative_genomics/jupyter/circos
bin/circos -conf /home/jovyan/training_bacterial_comparative_genomics/jupyter/circos_test.conf

(base) debug: using configuration file input /home/jovyan/training_bacterial_comparative_genomics/jupyter/circos_test.conf
debuglib: using configuration input /home/jovyan/training_bacterial_comparative_genomics/jupyter/circos_test.conf
debuglib: loading configuration from file /home/jovyan/training_bacterial_comparative_genomics/jupyter/circos_test.conf
debugload: using file from argument /home/jovyan/training_bacterial_comparative_genomics/jupyter/circos_test.conf
debugfile: looking for configuration file /home/jovyan/training_bacterial_comparative_genomics/jupyter/circos_test.conf
debugfile: found configuration file /home/jovyan/training_bacterial_comparative_genomics/jupyter/circos_test.conf
zoomregion ideogram 0 chr Chr         0   4600000 scale  1.00 absolutescale  1.00
registering tag Chr
ideogramspacing Chr Chr Chr Chr 115.000025
ideogramreport   0   Chr   0   Chr      0.000   4600.000   4600.001       0.000       0.000 r 1350 1340 1350 10
drawing ticks Chr radius 1350 type abs

: 1

In [141]:
cp -rf circos.png circos1.png

(base) 

: 1

In [142]:
cd /home/jovyan/training_bacterial_comparative_genomics/jupyter

(base) 

: 1

<img src="../images/circos1.svg" align="center" width="50%" style="display: block; margin: auto;"/> 

## Retrieve public genomes available for comparison

Public genomes can be retrived using NCBI : https://www.ncbi.nlm.nih.gov/datasets/genomes/

In [20]:
wget https://ftp.ncbi.nlm.nih.gov/genomes/GENOME_REPORTS/prokaryotes.txt

--2022-06-10 15:26:58--  https://ftp.ncbi.nlm.nih.gov/genomes/GENOME_REPORTS/prokaryotes.txt
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.13, 130.14.250.11, 2607:f220:41e:250::11, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.13|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 130422440 (124M) [text/plain]
Saving to: ‘prokaryotes.txt’

prokaryotes.txt     100%[===================>] 124.38M  20.1MB/s    in 7.5s    

2022-06-10 15:27:06 (16.6 MB/s) - ‘prokaryotes.txt’ saved [130422440/130422440]



In [21]:
grep "Xanthomonas oryzae pv. oryzae" prokaryotes.txt | grep -c 'chromosome:'

95


In [22]:
grep "Xanthomonas oryzae pv. oryzicola" prokaryotes.txt | grep -c 'chromosome:'

17


#### Retrieve 3 Xoc genomes (fasta and annotations)

In [23]:
grep "Xanthomonas oryzae pv. oryzicola" prokaryotes.txt | grep 'chromosome:' | head -3

Xanthomonas oryzae pv. oryzicola	129394	PRJNA248159	248159	Proteobacteria	Gammaproteobacteria	4.43083	64.1	chromosome:NZ_CP007810.1/CP007810.1	-	1	4137	3486	2015/06/08	2021/11/23	Complete Genome	Key Laboratory of Agricultural Biodiversity for Plant Disease Management under the Ministry of Education	SAMN02793996	GCA_001021915.1	REPR	ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/001/021/915/GCA_001021915.1_ASM102191v1	-	YM15
Xanthomonas oryzae pv. oryzicola	129394	PRJNA237971	237971	Proteobacteria	Gammaproteobacteria	5.0801	64	chromosome:NZ_CP007221.1/CP007221.1	-	1	4690	3924	2015/03/04	2022/01/21	Complete Genome	Cornell University	SAMN02640212	GCA_000940825.1	-	ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000/940/825/GCA_000940825.1_ASM94082v1	27148456	CFBP7342
Xanthomonas oryzae pv. oryzicola	129394	PRJNA280380	280380	Proteobacteria	Gammaproteobacteria	5.01777	63.9	chromosome:NZ_CP011959.1/CP011959.1	-	1	4564	3815	2015/06/30	2021/11/23	Complete Genome	Bogdanove Lab, Plant Pathology and Plan

In [165]:
wget https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/001/021/915/GCA_001021915.1_ASM102191v1/GCA_001021915.1_ASM102191v1_genomic.fna.gz -O Xoc_GCA_001021915.fna.gz
wget https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000/940/825/GCA_000940825.1_ASM94082v1/GCA_000940825.1_ASM94082v1_genomic.fna.gz -O Xoc_GCA_000940825.fna.gz
wget https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/001/042/835/GCA_001042835.1_ASM104283v1/GCA_001042835.1_ASM104283v1_genomic.fna.gz -O Xoc_GCA_001042835.fna.gz

--2022-06-13 13:20:04--  https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/001/021/915/GCA_001021915.1_ASM102191v1/GCA_001021915.1_ASM102191v1_genomic.fna.gz
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.12, 130.14.250.10, 2607:f220:41f:250::230, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.12|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1281033 (1.2M) [application/x-gzip]
Saving to: ‘Xoc_GCA_001021915.fna.gz’

Xoc_GCA_001021915.f 100%[===================>]   1.22M  2.23MB/s    in 0.5s    

2022-06-13 13:20:05 (2.23 MB/s) - ‘Xoc_GCA_001021915.fna.gz’ saved [1281033/1281033]

(base) --2022-06-13 13:20:05--  https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000/940/825/GCA_000940825.1_ASM94082v1/GCA_000940825.1_ASM94082v1_genomic.fna.gz
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.10, 130.14.250.12, 2607:f220:41e:250::11, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.10|

: 1

In [95]:
wget https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/001/021/915/GCA_001021915.1_ASM102191v1/GCA_001021915.1_ASM102191v1_genomic.gbff.gz -O Xoc_GCA_001021915.gbff.gz
wget https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000/940/825/GCA_000940825.1_ASM94082v1/GCA_000940825.1_ASM94082v1_genomic.gbff.gz -O Xoc_GCA_000940825.gbff.gz
wget https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/001/042/835/GCA_001042835.1_ASM104283v1/GCA_001042835.1_ASM104283v1_genomic.gbff.gz -O Xoc_GCA_001042835.gbff.gz

--2022-06-13 05:02:31--  https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/001/021/915/GCA_001021915.1_ASM102191v1/GCA_001021915.1_ASM102191v1_genomic.gbff.gz
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.7, 165.112.9.229, 2607:f220:41e:250::7, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.7|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2951060 (2.8M) [application/x-gzip]
Saving to: ‘Xoc_GCA_001021915.gbff.gz’

Xoc_GCA_001021915.g 100%[===================>]   2.81M  4.33MB/s    in 0.6s    

2022-06-13 05:02:32 (4.33 MB/s) - ‘Xoc_GCA_001021915.gbff.gz’ saved [2951060/2951060]

(base) --2022-06-13 05:02:32--  https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000/940/825/GCA_000940825.1_ASM94082v1/GCA_000940825.1_ASM94082v1_genomic.gbff.gz
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.7, 165.112.9.229, 2607:f220:41e:250::10, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.7|:4

: 1

#### Retrieve 3 Xoo genomes (fasta and annotations)

In [24]:
grep "Xanthomonas oryzae pv. oryzae" prokaryotes.txt | grep 'chromosome:' | head -3

Xanthomonas oryzae pv. oryzae	64187	PRJNA485465	485465	Proteobacteria	Gammaproteobacteria	4.99067	63.7	chromosome:NZ_CP031697.1/CP031697.1	-	1	4647	3652	2019/02/04	2022/03/27	Complete Genome	Martin Luther University Halle-Wittenberg	SAMN09791860	GCA_004136375.1	-	ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/004/136/375/GCA_004136375.1_ASM413637v1	-	ICMP3125
Xanthomonas oryzae pv. oryzae	64187	PRJNA269487	269487	Proteobacteria	Gammaproteobacteria	5.09305	63.7	chromosome:NZ_CP040687.1/CP040687.1	-	1	4757	3787	2019/05/28	2022/03/20	Complete Genome	Bacterial Genomics and Evolution Laboratory, CSIR-Institute of Microbial Technology, Sector 39-A, Chandigarh, India	SAMN03252586	GCA_001929235.2	-	ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/001/929/235/GCA_001929235.2_ASM192923v2	-	IXO1088
Xanthomonas oryzae pv. oryzae	64187	PRJNA497605	497605	Proteobacteria	Gammaproteobacteria	5.05039	63.7	chromosome:NZ_CP033192.1/CP033192.1	-	1	4713	3695	2019/03/19	2022/04/07	Complete Genome	University of Florid

In [164]:
wget https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/004/136/375/GCA_004136375.1_ASM413637v1/GCA_004136375.1_ASM413637v1_genomic.fna.gz -O Xoo_GCA_004136375.fna.gz
wget https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/001/929/235/GCA_001929235.2_ASM192923v2/GCA_001929235.2_ASM192923v2_genomic.fna.gz -O Xoo_GCA_001929235.fna.gz
wget https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/004/355/825/GCA_004355825.1_ASM435582v1/GCA_004355825.1_ASM435582v1_genomic.fna.gz -O Xoo_GCA_004355825.fna.gz

--2022-06-13 13:19:47--  https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/004/136/375/GCA_004136375.1_ASM413637v1/GCA_004136375.1_ASM413637v1_genomic.fna.gz
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.10, 130.14.250.12, 2607:f220:41e:250::11, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1415838 (1.3M) [application/x-gzip]
Saving to: ‘Xoo_GCA_004136375.fna.gz’

Xoo_GCA_004136375.f 100%[===================>]   1.35M  2.17MB/s    in 0.6s    

2022-06-13 13:19:49 (2.17 MB/s) - ‘Xoo_GCA_004136375.fna.gz’ saved [1415838/1415838]

(base) --2022-06-13 13:19:49--  https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/001/929/235/GCA_001929235.2_ASM192923v2/GCA_001929235.2_ASM192923v2_genomic.fna.gz
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.10, 130.14.250.12, 2607:f220:41f:250::230, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.1

: 1

In [97]:
wget https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/004/136/375/GCA_004136375.1_ASM413637v1/GCA_004136375.1_ASM413637v1_genomic.gbff.gz -O Xoo_GCA_004136375.gbff.gz
wget https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/001/929/235/GCA_001929235.2_ASM192923v2/GCA_001929235.2_ASM192923v2_genomic.gbff.gz -O Xoo_GCA_001929235.gbff.gz
wget https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/004/355/825/GCA_004355825.1_ASM435582v1/GCA_004355825.1_ASM435582v1_genomic.gbff.gz -O Xoo_GCA_004355825.gbff.gz

--2022-06-13 06:58:10--  https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/004/136/375/GCA_004136375.1_ASM413637v1/GCA_004136375.1_ASM413637v1_genomic.gbff.gz
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 165.112.9.228, 130.14.250.12, 2607:f220:41e:250::11, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|165.112.9.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3276284 (3.1M) [application/x-gzip]
Saving to: ‘Xoo_GCA_004136375.gbff.gz’

Xoo_GCA_004136375.g 100%[===================>]   3.12M  4.31MB/s    in 0.7s    

2022-06-13 06:58:11 (4.31 MB/s) - ‘Xoo_GCA_004136375.gbff.gz’ saved [3276284/3276284]

(base) --2022-06-13 06:58:11--  https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/001/929/235/GCA_001929235.2_ASM192923v2/GCA_001929235.2_ASM192923v2_genomic.gbff.gz
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 165.112.9.228, 130.14.250.12, 2607:f220:41f:250::229, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|165.112.

: 1

In [166]:
gunzip *.fna.gz

(base) 

: 1

## Convert GenBank annotation files to GFF format

In [99]:
rm -rf annotations
mkdir annotations

(base) (base) 

: 1

In [100]:
cp -rf Xo*gbff.gz annotations

(base) 

: 1

In [101]:
git clone https://github.com/bioperl/bioperl-live.git
chmod 755 bioperl-live/bin/*

fatal: destination path 'bioperl-live' already exists and is not an empty directory.
(base) (base) 

: 1

In [103]:
perl bioperl-live/bin/bp_genbank2gff3 annotations/Xoo_GCA_004136375.gbff.gz
perl bioperl-live/bin/bp_genbank2gff3 annotations/Xoo_GCA_001929235.gbff.gz
perl bioperl-live/bin/bp_genbank2gff3 annotations/Xoo_GCA_004355825.gbff.gz
perl bioperl-live/bin/bp_genbank2gff3 annotations/Xoc_GCA_001021915.gbff.gz
perl bioperl-live/bin/bp_genbank2gff3 annotations/Xoc_GCA_000940825.gbff.gz
perl bioperl-live/bin/bp_genbank2gff3 annotations/Xoc_GCA_001042835.gbff.gz

# Input: annotations/Xoo_GCA_004136375.gbff.gz
# working on region:CP031697, Xanthomonas oryzae pv. oryzae, Xanthomonas oryzae pv. oryzae strain ICMP3125 chromosome, complete genome., 04-FEB-2019
# GFF3 saved to ./Xoo_GCA_004136375.gbff.gz.gff
(base) # Input: annotations/Xoo_GCA_001929235.gbff.gz
# working on region:CP040687, Xanthomonas oryzae pv. oryzae, Xanthomonas oryzae pv. oryzae strain IXO1088 chromosome, complete genome., 06-MAR-2020
# GFF3 saved to ./Xoo_GCA_001929235.gbff.gz.gff
(base) # Input: annotations/Xoo_GCA_004355825.gbff.gz
# working on region:CP033192, Xanthomonas oryzae pv. oryzae, Xanthomonas oryzae pv. oryzae strain NX0260 chromosome, complete genome., 19-MAR-2019
# GFF3 saved to ./Xoo_GCA_004355825.gbff.gz.gff
(base) # Input: annotations/Xoc_GCA_001021915.gbff.gz
# working on region:CP007810, Xanthomonas oryzae pv. oryzicola, Xanthomonas oryzae pv. oryzicola strain YM15, complete genome., 08-JUN-2015
# GFF3 saved to ./Xoc_GCA_001021915.gbff.gz.gff
(base) # Input:

: 1

In [104]:
mv Xo*gff annotations

(base) 

: 1

## Comparison between 2 genomes (with Minimap2 and circos)

#### Compare 2 Xoo

In [145]:
gunzip Xoo*fna.gz

(base) 

: 1

In [146]:
ls Xoo*fna

Xoo_GCA_001929235.fna  Xoo_GCA_004136375.fna  Xoo_GCA_004355825.fna
(base) 

: 1

In [176]:
minimap2 Xoo_GCA_001929235.fna Xoo_GCA_004136375.fna -o minimap2.paf

[M::mm_idx_gen::0.268*1.00] collected minimizers
[M::mm_idx_gen::0.413*0.99] sorted minimizers
[M::main::0.413*0.99] loaded/built the index for 1 target sequence(s)
[M::mm_mapopt_update::0.437*0.98] mid_occ = 71
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 1
[M::mm_idx_stat::0.456*0.97] distinct minimizers: 780094 (93.86% are singletons); average occurrences: 1.219; average spacing: 5.354
[M::worker_pipeline::21.876*0.98] mapped 1 sequences
[M::main] Version: 2.17-r941
[M::main] CMD: minimap2 -o minimap2.paf Xoo_GCA_001929235.fna Xoo_GCA_004136375.fna
[M::main] Real time: 21.898 sec; CPU: 21.369 sec; Peak RSS: 0.293 GB
(base) 

: 1

In [177]:
awk {'if (($4-$3)>100000 && $5=="+")print "link"NR-1 " "$1" "$3" "$4'} minimap2.paf >links.txt
awk {'if (($4-$3)>100000 && $5=="+")print "link"NR-1 " "$6" "$8" "$9'} minimap2.paf >>links.txt

(base) (base) 

: 1

In [178]:
awk {'if (($4-$3)>100000 && $5=="-")print "link"NR-1 " "$1" "$3" "$4'} minimap2.paf >links_inverted.txt
awk {'if (($4-$3)>100000 && $5=="-")print "link"NR-1 " "$6" "$8" "$9'} minimap2.paf >>links_inverted.txt

(base) (base) 

: 1

#### Visualize matches between genomes with Circos

In [149]:
cd /home/jovyan/training_bacterial_comparative_genomics/jupyter/circos
cp -rf ../../data/circos2.conf circos2.conf

(base) (base) 

: 1

Edit the Circos configuration file to add informations about links
```
<link segdup-bundle34>
z            = 50
color        = nyt_red
thickness    = 3
file         = /home/jovyan/training_bacterial_comparative_genomics/jupyter/links.txt
bezier_radius_purity = 0.2
ribbon      = yes
crest = 1
</link>

<link segdup-bundle35>
z            = 50
color        = nyt_blue
thickness    = 3
file         = /home/jovyan/training_bacterial_comparative_genomics/jupyter/links_inverted.txt
bezier_radius_purity = 0.2
ribbon      = yes
crest = 1
</link>
```

Edit a karyotype file with the two genomes and their size

Make one of the genome in the opposite way by entering this line in the Circos configuration file

chromosomes_reverse = CP040687.1

In [150]:
bin/circos -conf circos2.conf

debug: using configuration file input circos2.conf
debuglib: using configuration input circos2.conf
debuglib: loading configuration from file circos2.conf
debugload: using file from argument circos2.conf
debugfile: looking for configuration file circos2.conf
debugfile: found configuration file circos2.conf
Config::General The file "etc/image.conf" does not exist within ConfigPath: /home/jovyan/training_bacterial_comparative_genomics/jupyter/circos/bin/etc./home/jovyan/training_bacterial_comparative_genomics/jupyter/circos/bin/../etc./home/jovyan/training_bacterial_comparative_genomics/jupyter/circos/bin/.../home/jovyan/training_bacterial_comparative_genomics/jupyter/circos/bin.../home/jovyan/training_bacterial_comparative_genomics/jupyter/circos/bin/../.! at /home/jovyan/training_bacterial_comparative_genomics/jupyter/circos/bin/../lib/Circos.pm line 9135.
(base) 

: 1

<img src="../images/circos2.png" align="center" width="70%" style="display: block; margin: auto;"/>

#### Do the same process for the comparing one Xoo and one Xoc

## Compare genome similarity using Average Nucleotide Identity (ANI) (fastANI)

In [167]:
ls Xo*fna >list_genomes.txt

(base) 

: 1

In [168]:
fastANI --rl list_genomes.txt --ql list_genomes.txt -o fastani.out --matrix

>>>>>>>>>>>>>>>>>>
Reference = [Xoc_GCA_000940825.fna, Xoc_GCA_001021915.fna, Xoc_GCA_001042835.fna, Xoo_GCA_001929235.fna, Xoo_GCA_004136375.fna, Xoo_GCA_004355825.fna]
Query = [Xoc_GCA_000940825.fna, Xoc_GCA_001021915.fna, Xoc_GCA_001042835.fna, Xoo_GCA_001929235.fna, Xoo_GCA_004136375.fna, Xoo_GCA_004355825.fna]
Kmer size = 16
Fragment length = 3000
Threads = 1
ANI output file = fastani.out
>>>>>>>>>>>>>>>>>>
INFO [thread 0], skch::main, Count of threads executing parallel_for : 1
INFO [thread 0], skch::Sketch::build, window size for minimizer sampling  = 24
INFO [thread 0], skch::Sketch::build, minimizers picked from reference = 2380733
INFO [thread 0], skch::Sketch::index, unique minimizers = 524575
INFO [thread 0], skch::Sketch::computeFreqHist, Frequency histogram of minimizers = (1, 69292) ... (1492, 1)
INFO [thread 0], skch::Sketch::computeFreqHist, consider all minimizers during lookup.
INFO [thread 0], skch::main, Time spent sketching the reference : 3.11525 sec
INFO [thread

: 1

##### Check that genomes are closer within the same pathovar than between pathovars.

##### Which are the closest genomes?

In [169]:
more fastani.out.matrix

6
Xoc_GCA_000940825.fna
Xoc_GCA_001021915.fna	99.149597
Xoc_GCA_001042835.fna	99.411034	99.196701
Xoo_GCA_001929235.fna	97.226959	97.219826	97.288483
Xoo_GCA_004136375.fna	97.313660	97.230530	97.365417	99.582520
Xoo_GCA_004355825.fna	97.236938	97.235214	97.313324	99.704956	99.590271
(base) 

: 1

##### How to visualize ANI as heatmap matrix?

Install using the R interface the library optparse and heatmaply

install.packages("optparse")

install.packages("heatmaply")

In [170]:
perl ../convertANI.pl fastani.out.matrix >fastani.out.matrix.completed

(base) 

: 1

In [171]:
Rscript ../heatmaply.R -f fastani.out.matrix.completed

Loading required package: plotly
Loading required package: ggplot2

Attaching package: ‘plotly’

The following object is masked from ‘package:ggplot2’:

    last_plot

The following object is masked from ‘package:stats’:

    filter

The following object is masked from ‘package:graphics’:

    layout

Loading required package: viridis
Loading required package: viridisLite

Welcome to heatmaply version 1.3.0

Type citation('heatmaply') for how to cite the package.
Type ?heatmaply for the main documentation.

The github page is: https://github.com/talgalili/heatmaply/
Please submit your suggestions and bug-reports at: https://github.com/talgalili/heatmaply/issues
You may ask questions at stackoverflow, use the r and heatmaply tags: 
	 https://stackoverflow.com/questions/tagged/heatmaply

(base) 

: 1

##### Try to make the same process including our newly assembled genome

## Gene content comparison / Pangenome analysis (using Roary)

In [129]:
rm -rf roary_out
roary -f roary_out annotations/*gff

(base) Use of uninitialized value in require at /usr/lib/x86_64-linux-gnu/perl5/5.30/Encode.pm line 70.

Please cite Roary if you use any of the results it produces:
    Andrew J. Page, Carla A. Cummins, Martin Hunt, Vanessa K. Wong, Sandra Reuter, Matthew T. G. Holden, Maria Fookes, Daniel Falush, Jacqueline A. Keane, Julian Parkhill,
	"Roary: Rapid large-scale prokaryote pan genome analysis", Bioinformatics, 2015 Nov 15;31(22):3691-3693
    doi: http://doi.org/10.1093/bioinformatics/btv421
	Pubmed: 26198102

2022/06/13 08:51:34 Input file contains duplicate gene IDs, attempting to fix by adding a unique suffix, new GFF in the fixed_input_files directory: /home/jovyan/training_bacterial_comparative_genomics/jupyter/annotations/Xoo_GCA_001929235.gbff.gz.gff 
2022/06/13 08:51:36 Input file contains duplicate gene IDs, attempting to fix by adding a unique suffix, new GFF in the fixed_input_files directory: /home/jovyan/training_bacterial_comparative_genomics/jupyter/annotations/Xoo_GCA_0

: 1

In [130]:
head -10 roary_out/gene_presence_absence.csv

"Gene","Non-unique Gene name","Annotation","No. isolates","No. sequences","Avg sequences per isolate","Genome Fragment","Order within Fragment","Accessory Fragment","Accessory Order with Fragment","QC","Min group size nuc","Max group size nuc","Avg group size nuc","Xoc_GCA_000940825.gbff.gz","Xoc_GCA_001021915.gbff.gz","Xoc_GCA_001042835.gbff.gz","Xoo_GCA_001929235.gbff.gz","Xoo_GCA_004136375.gbff.gz","Xoo_GCA_004355825.gbff.gz"
"IXO1088_008755","","response regulator","6","6","1","1","1683","","","","404","404","404","BE73_06245.p01","FE36_14125.p01","ACU17_06495.p01","10bd397a0f88aca11976f4a4f6631125_6321","4e95a45ca635958849417c3d368d746d_7522","cc532a0030968c6887cea9dec5a101c0_11970"
"IXO1088_008760","","hypothetical protein","6","6","1","1","1682","","","","497","497","497","BE73_06250.p01","FE36_14120.p01","ACU17_06500.p01","10bd397a0f88aca11976f4a4f6631125_6325","4e95a45ca635958849417c3d368d746d_7526","cc532a0030968c6887cea9dec5a101c0_11966"
"EBA21_16420","","hypothetical protei

: 1

In [131]:
more roary_out/summary_statistics.txt

Core genes	(99% <= strains <= 100%)	2565
Soft core genes	(95% <= strains < 99%)	0
Shell genes	(15% <= strains < 95%)	3763
Cloud genes	(0% <= strains < 15%)	0
Total genes	(0% <= strains <= 100%)	6328
(base) 

: 1

In [92]:
git clone https://github.com/sanger-pathogens/Roary.git

Cloning into 'Roary'...
remote: Enumerating objects: 7454, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 7454 (delta 0), reused 1 (delta 0), pack-reused 7449
Receiving objects: 100% (7454/7454), 18.12 MiB | 6.84 MiB/s, done.
Resolving deltas: 100% (4181/4181), done.
(base) 

: 1

In [132]:
python Roary/contrib/roary_plots/roary_plots.py roary_out/accessory_binary_genes.fa.newick roary_out/gene_presence_absence.csv

(base) 

: 1

In [ ]:
query_pan_genome -g roary_out/clustered_proteins -a difference --input_set_one annotations/Xoc_GCA_000940825.gbff.gz.gff,annotations/Xoc_GCA_001021915.gbff.gz.gff,annotations/Xoc_GCA_001021915.gbff.gz.gff --input_set_two annotations/Xoo_GCA_001929235.gbff.gz.gff,annotations/Xoo_GCA_004136375.gbff.gz.gff,annotations/Xoo_GCA_004355825.gbff.gz.gff

<img src="../images/pangenome_pie.png" align="center" width="30%" style="display: block; margin: auto;"/>

<img src="../images/pangenome_matrix.png" align="center" width="50%"/>

## Association tests with metadata (Pan-GWAS)

In [118]:
cp -rf ../data/pangenome_100xantho ./
cd pangenome_100xantho

(base) (base) 

: 1

In [119]:
scoary -t traits.txt -g gene_presence_absence.csv

==== Scoary started ====
Command: /usr/bin/scoary -t traits.txt -g gene_presence_absence.csv
Reading gene presence absence file
Creating Hamming distance matrix based on gene presence/absence
Building UPGMA tree from distance matrix
Reading traits file
ERROR: Some isolates in your gene presence absence file were not represented in your traits file. These will count as MISSING data and will not be included.
Finished loading files into memory.


==== Performing statistics ====
-- Filtration options --
Individual (Naive):    0.05
Collapse genes:    False


Tallying genes and performing statistical analyses
Gene-wise counting and Fisher's exact tests for trait: pathovar
100.00%
Adding p-values adjusted for testing multiple hypotheses

Storing results: pathovar
Calculating max number of contrasting pairs for each nominally significant gene
100.00%
Storing results to file


==== Finished ====
Checked a total of 13554 genes for associations to 1 trait(s). Total time used: 24 seconds.
Scoary f

: 1

#### How many clusters are specifically present in pathovar oryzae (absent in oryzicola)

In [120]:
awk -F "\",\"" {'if ($4==0 && $5==75)print $_'} pathovar_*.results.csv | wc -l

28
(base) 

: 1

#### How many clusters are specifically present in pathovar oryzicola (absent in oryzae)

In [121]:
awk -F "\",\"" {'if ($4==15 && $5==0)print $_'} pathovar_*.results.csv | wc -l

110
(base) 

: 1